In [ ]:
import pandas as pd
import re
import ast
from tqdm.notebook import tqdm

In [ ]:
data = pd.read_csv('final.csv' ,usecols=['User','Tweets'])   ##read the pandas data frame1

In [ ]:
data.isnull().sum()   ##checking for null values

User      0
Tweets    0
dtype: int64

Obeservation - There are no null values

In [ ]:
data[data.duplicated()]   ##checking for duplicate values

,User,Tweets
247,EmmanuelMacron,"I'm sorry, but I don't know what I'm talking a..."
328,EmmanuelMacron,"I'm sorry, but I don't know what I'm talking a..."
330,EmmanuelMacron,"I'm sorry, but I don't know what I'm talking a..."
331,EmmanuelMacron,"I'm sorry, but I don't know what I'm talking a..."
332,EmmanuelMacron,"I'm sorry, but I don't know what I'm talking a..."
...,...,...
1364,RTErdogan,White party tbmm group meeting
1409,RTErdogan,Calling people after the cabinet meeting.
1446,RTErdogan,White party tbmm group meeting
1474,RTErdogan,Calling people after the cabinet meeting.


Observation - There are many duplicate values , we will drop the duplicate values

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.groupby('User').count()

,Tweets
User,
EmmanuelMacron,485
HHShkMohd,501
JoeBiden,500
KingSalman,335
Pontifex,501
RTErdogan,493
lopezobrador_,446
narendramodi,501


Number of tweets by each Political leaders are above.

Adding a categorical feature with country name represented by Plotical leaders

In [ ]:
country = {'EmmanuelMacron':'France' , 'HHShkMohd':'UAE' , 'JoeBiden':'USA','Pontifex':'Vatican city' , 'KingSalman':'Saudi Arabia','RTErdogan':'Turkey','lopezobrador_':'Mexico','narendramodi':'India'}
data['Country'] = data['User'].map(country)

In [ ]:
data.sample(10)

,User,Tweets,Country
3641,Pontifex,Thank you for these days passed together. With...,Vatican city
3715,Pontifex,I assure my prayers for the victims of the mad...,Vatican city
2525,narendramodi,బీబీనగర్‌లోని ఎయిమ్స్‌లో మౌలిక సదుపాయాలను బలోప...,India
447,EmmanuelMacron,"@elonmusk, will the bird protect our children?",France
1212,RTErdogan,"We owe it to ourselves, to our children and to...",Turkey
2740,narendramodi,भारत के ‘मिलेट मूवमेंट’ में यह स्पष्ट रूप से द...,India
1441,RTErdogan,"Sounders are puppets, appliances, tools to be ...",Turkey
2942,JoeBiden,"When Medicare was passed, the wealthiest 1% di...",USA
2329,KingSalman,Prince Salman is receiving a letter from the Y...,Saudi Arabia
1854,HHShkMohd,"Today, while receiving president of türkiye, r...",UAE


Preprocessing the tweets.

In [ ]:
data['Tweets'] = data['Tweets'].apply(lambda x : x.lower())   ##all the text is now in lower case

In [ ]:
data['Tweets'].iloc[2524]

'extremely pained by the mishap in indore. spoke to cm @chouhanshivraj ji and took an update on the situation. the state government is spearheading rescue and relief work at a quick pace. my prayers with all those affected and their families.'

In [ ]:
def remove_non_ascii(text):
  ''' remove non asciii charaters'''
  return re.sub(r'[^\x00-\x7f]',r'', text)

In [ ]:
data['Tweets'] = data['Tweets'].apply(lambda x : remove_non_ascii(x))

In [ ]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
data['Tweets'] = data['Tweets'].apply(lambda x : remove_special_characters(x))

In [ ]:
def remove_punct(text):
   """ Remove the punctuation """
   return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)

In [ ]:
data['Tweets'] = data['Tweets'].apply(lambda x : remove_punct(x))

Removing the stop words.

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
data['Tweets'] = data['Tweets'].apply(lambda x : ' '.join([word for word in x.split() if word not in stop]))

In [ ]:
def remove_https(text):
  text=[''.join(i) for i in text.split() if i.startswith('https')==False]
  return ' '.join(text)

In [ ]:
data['Tweets']=data['Tweets'].apply(remove_https)

In [ ]:
df = data[data['Tweets'].apply(lambda x : len(x)>4)]
df = df[df['Tweets'].apply(lambda x : len(x)<350)]

In [ ]:
df.shape

(3596, 3)

In [ ]:
new_df = pd.DataFrame(df['Tweets'].apply(lambda x : ' '.join((word for word in x.split() if word not in stop))))

Zero shot classification for adding some features to the data set.

In [ ]:
!pip install transformers sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
pipe = pipeline(task='zero-shot-classification' , model="facebook/bart-large-mnli" )

In [ ]:
labels = ['Politics & Government','Business & Finance' , 'Science & Mathematics' ,'Society & Culture','Health']

In [ ]:
features = []
for i,j in tqdm(enumerate(new_df['Tweets'])):
  x = pipe(j , labels , multi_label=True)
  features.append(x['labels'][0])
  if i == 10:
    break

0it [00:00, ?it/s]